## Загружаем необходимые библиотеки

In [1]:
import os

import random
import pandas as pd
import numpy as np
import shutil
from tqdm import tqdm
import time

import matplotlib
import plotly
import PIL
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from google.colab.patches import cv2_imshow

from torch.utils.data import Dataset, DataLoader
import glob, matplotlib.pyplot as plt, matplotlib.patches as patches

import yaml
import torch
import json
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from pathlib import Path
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from sklearn.model_selection import train_test_split

## Загрузка датасета




In [2]:
!git clone 'https://github.com/MelnikDM/SVO.git'

Cloning into 'SVO'...
remote: Enumerating objects: 13803, done.
remote: Counting objects: 100% (2871/2871), done.
remote: Compressing objects: 100% (1462/1462), done.
remote: Total 13803 (delta 1420), reused 2852 (delta 1409), pack-reused 10932
Receiving objects: 100% (13803/13803), 947.35 MiB | 33.20 MiB/s, done.
Resolving deltas: 100% (7062/7062), done.
Updating files: 100% (4920/4920), done.


## Обработка аннотаций

И разбиение выборок на обучающую, валидационную и тестовую





In [3]:
image_list = os.listdir('/content/SVO/LA_ObjectDetection/data/Images')
train_list, test_list = train_test_split(image_list, test_size=0.2)
val_list, test_list = train_test_split(test_list, test_size=0.5)

print('Обучающая выборка: ',len(train_list))
print('Проверочная выборка: ',len(val_list))
print('Тестовая выборка:' ,len(test_list))

Обучающая выборка:  1799
Проверочная выборка:  225
Тестовая выборка: 225


In [4]:
!mkdir '/content/SVO/LA_ObjectDetection/SSD300_VGG16/data/'
!mkdir '/content/SVO/LA_ObjectDetection/SSD300_VGG16/data/LA'

In [5]:
!mkdir '/content/SVO/LA_ObjectDetection/SSD300_VGG16/data/LA/train'
!mkdir '/content/SVO/LA_ObjectDetection/SSD300_VGG16/data/LA/valid'
!mkdir '/content/SVO/LA_ObjectDetection/SSD300_VGG16/data/LA/test'

In [6]:
!mkdir '/content/outputs/'

In [7]:
def move_data(name_list, folder, newfolder, mode, img=True):
  """
  Функция перемещает файлы в указанную папку.
  name_list - список с названиями файлов для перемещения
  folder    - путь к папке откуда берем файл
  mode      - название папки куда перемещаем
  img       - указывает какой тип файла мы перемещаем
  """
  format = 'jpg' if img==True else 'xml'
  for name in name_list:
    name = name[:-3]+format
    shutil.move(folder + name, newfolder + '{}/'.format(mode) + name)

In [8]:
move_data(train_list, '/content/SVO/LA_ObjectDetection/data/Images/', '/content/SVO/LA_ObjectDetection/SSD300_VGG16/data/LA/', 'train', img=True)
move_data(val_list, '/content/SVO/LA_ObjectDetection/data/Images/', '/content/SVO/LA_ObjectDetection/SSD300_VGG16/data/LA/', 'valid', img=True)

In [9]:
move_data(train_list, '/content/SVO/LA_ObjectDetection/data/Annotations/', '/content/SVO/LA_ObjectDetection/SSD300_VGG16/data/LA/', 'train', img=False)
move_data(val_list, '/content/SVO/LA_ObjectDetection/data/Annotations/', '/content/SVO/LA_ObjectDetection/SSD300_VGG16/data/LA/', 'valid', img=False)

In [11]:
move_data(test_list, '/content/SVO/LA_ObjectDetection/data/Images/', '/content/SVO/LA_ObjectDetection/SSD300_VGG16/data/LA/', 'test', img=True)

## Обучение модели

In [14]:
!python /content/SVO/LA_ObjectDetection/SSD300_VGG16/datasets.py

Number of training images: 1799
qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.10/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [16]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 4.7 MB/s eta 0:00:00


In [17]:
!python /content/SVO/LA_ObjectDetection/SSD300_VGG16/train.py

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Number of training samples: 1799
Number of validation samples: 1799

Downloading: "https://download.pytorch.org/models/ssd300_vgg16_coco-b556d3b4.pth" to /root/.cache/torch/hub/checkpoints/ssd300_vgg16_coco-b556d3b4.pth
100% 136M/136M [00:00<00:00, 155MB/s]
SSD(
  (backbone): SSDFeatureExtractorVGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): Re

In [28]:
!python /content/SVO/LA_ObjectDetection/SSD300_VGG16/inference.py --input /content/SVO/LA_ObjectDetection/SSD300_VGG16/data/LA/test --imgsz 640

Test instances: 225
(640, 640, 3)
Image 1 done...
--------------------------------------------------
(640, 640, 3)
Image 2 done...
--------------------------------------------------
(640, 640, 3)
Image 3 done...
--------------------------------------------------
(640, 640, 3)
Image 4 done...
--------------------------------------------------
(640, 640, 3)
Image 5 done...
--------------------------------------------------
(640, 640, 3)
Image 6 done...
--------------------------------------------------
(640, 640, 3)
Image 7 done...
--------------------------------------------------
(640, 640, 3)
Image 8 done...
--------------------------------------------------
(640, 640, 3)
Image 9 done...
--------------------------------------------------
(640, 640, 3)
Image 10 done...
--------------------------------------------------
(640, 640, 3)
Image 11 done...
--------------------------------------------------
(640, 640, 3)
Image 12 done...
--------------------------------------------------
(640,

In [29]:
!zip -r /content/FRCNN_1.zip /content/inference_outputs/images

  adding: content/inference_outputs/images/ (stored 0%)
  adding: content/inference_outputs/images/LA0230_jpg.jpg (deflated 5%)
  adding: content/inference_outputs/images/LA0615_jpg.jpg (deflated 7%)
  adding: content/inference_outputs/images/LA0310_jpg.jpg (deflated 4%)
  adding: content/inference_outputs/images/LA0187_jpg.jpg (deflated 4%)
  adding: content/inference_outputs/images/LA0445_jpg.jpg (deflated 7%)
  adding: content/inference_outputs/images/LA0223_jpg.jpg (deflated 4%)
  adding: content/inference_outputs/images/LA774_jpg.jpg (deflated 4%)
  adding: content/inference_outputs/images/LA0376_jpg.jpg (deflated 4%)
  adding: content/inference_outputs/images/LA0570_jpg.jpg (deflated 11%)
  adding: content/inference_outputs/images/LA0293_jpg.jpg (deflated 9%)
  adding: content/inference_outputs/images/LA0173_jpg.jpg (deflated 4%)
  adding: content/inference_outputs/images/LA0458_jpg.jpg (deflated 10%)
  adding: content/inference_outputs/images/LA798_jpg.jpg (deflated 8%)
  adding

In [30]:
from google.colab import files
files.download("/content/FRCNN_1.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
!python /content/SVO/LA_ObjectDetection/SSD300_VGG16/inference_video.py --input /content/SVO/LA_ObjectDetection/data/Videos/Leo2.mp4 --imgsz 640

Leo2
Average FPS: 57.397
